In [6]:
#LSTM-XGB-250
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_models():
    lstm_model = keras.models.load_model('Original-LSTM-XGB-Dataset-A-1-80-20-LSTM-Model.h5')
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-XGB-model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    with open('Original-LSTM-XGB-Dataset-A-1-80-20-class_names.pkl', 'rb') as f:
        class_names = pickle.load(f)
    return lstm_model, xgb_model, scaler, label_encoder, class_names
def predict_with_models(lstm_model, xgb_model, scaler, label_encoder, class_names, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_scaled = scaler.transform(features_df)

    # Reshape features_scaled to 3D for LSTM input
    features_scaled_reshaped = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))

    # Get intermediate output from LSTM
    intermediate_output = lstm_model.predict(features_scaled_reshaped)

    # Print the shapes to debug
   # print("Features scaled shape:", features_scaled.shape)
   # print("Intermediate output shape:", intermediate_output.shape)

    # Predict with XGBoost
    xgb_predictions = xgb_model.predict(intermediate_output)
    
    # Map predicted class indices to class names
    predicted_class_name = class_names[xgb_predictions[0]]
    
    return predicted_class_name

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict
def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0,
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
        'Total-Pkt': initial_total_packets,
        'T-Time': initial_total_time
    }

    return features, None  
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    pcap_file = 'Eyezy-100.pcap'
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    
    lstm_model, xgb_model, scaler, label_encoder, class_names = load_pretrained_models()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)

    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    
    

    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []

    sniff_packets = scapy.sniff(offline=pcap_file)
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 250 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)

                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        label_row = previous_samples[previous_samples['Bandwidth'] == initial_bandwidth]
                        if not label_row.empty:
                            label = label_row['Label'].values[0]
                        else:
                            label = 'Unknown'
                    else:
                        label = 'Normal Traffic'
                    true_labels.append(label)
                    predicted_label = predict_with_models(lstm_model, xgb_model, scaler, label_encoder, class_names, features)
                
               
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                
                
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
                
            
            captured_packets = []
            packet_count = 0

if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


C:\Users\mimi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Predicted Label: Eyezy
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


C:\Users\mimi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Predicted Label: Eyezy
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


...

C:\Users\mimi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Predicted Label: Eyezy
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


C:\Users\mimi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Predicted Label: Eyezy
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: mLite
Accuracy: 0.9565217391304348
Precision: 0.9580757391304349
Recall: 0.9565217391304348
F1-Score: 0.9572981084715032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.9583333333333334
Precision: 0.9598873333333334
Recall: 0.9583333333333334
F1-Score: 0.9591097038656079
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Label: Eyezy
Accuracy: 0.96
Precision: 0.961554
Recall: 0.96
F1-Score: 0.9607763716242167
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: Eyezy
Accuracy: 0.9615384615384616
Precision: 0.9630924615384616
Recall: 0.9615384615384616
F1-Score: 0.9623148341672677
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Label: KidsGuardPro
Accuracy: 0.92592592592

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

